In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#data preparation/pre-processing
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names= ['type', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['type','message'])
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

# encode dataset labels for model
all_data = pd.concat([train_data, test_data])
all_data['type'], label_mapping = pd.factorize(all_data['type'])
train_data['type'] = all_data['type'][:len(train_data)]
test_data['type'] = all_data['type'][len(train_data):]

# encoding messages
token = Tokenizer(num_words = 10000)
maxlen = 30
token.fit_on_texts(train_data['message'])

train_seqs = token.texts_to_sequences(train_data['message'])
train_padded_seqs = pad_sequences(train_seqs, maxlen=maxlen, padding='post', truncating='post')
test_seqs = token.texts_to_sequences(test_data['message'])
test_padded_seqs = pad_sequences(test_seqs, maxlen=maxlen, padding='post', truncating='post')

# since training and testing sets are given separate we can reshape data manually

train_labels = np.array(train_data['type'].tolist())
train_ds = np.array(train_padded_seqs.tolist())
test_labels = np.array(test_data['type'].tolist())
test_ds = np.array(test_padded_seqs.tolist())

print(test_labels)
print(test_ds)


In [ ]:
vocab_size = 100000
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

history = model.fit(
    train_ds,
    train_labels,
    validation_data=(test_ds,test_labels),
    validation_steps=30,
    epochs=5,
    batch_size=32
)

In [ ]:
test_loss, test_acc = model.evaluate(test_ds,test_labels)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)
# was testing class imbalance to see what error threshold was needed
#train_data['type'].value_counts()
#test_data['type'].value_counts()

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  seq = token.texts_to_sequences([pred_text])
  padded_seq = pad_sequences(seq, maxlen=maxlen, padding='post', truncating='post')
  print(padded_seq)
  prediction = model.predict(padded_seq)
  print(prediction)
  if prediction[0][0] < 0.5:
    return [prediction[0][0], 'ham']
  return [prediction[0][0], 'spam']

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
